<a href="https://colab.research.google.com/github/steinjm21/ColabInClass/blob/main/matplotlibProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/steinjm21/matplotlibProject.git

fatal: destination path 'matplotlibProject' already exists and is not an empty directory.


Defining function to be outputted to .csv

In [22]:
import numpy as np
import sys
import datetime
import math

In [23]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count

I found using np.empty and np.vstack to be more efficient (it actually worked) with the code I had written and tried to painstakingly edit for hours.

In [37]:
if __name__ == '__main__':
    grid = np.empty((0, 4))
    for n in range(100, 15001):
      n_samples = int(n)
      start_time = datetime.datetime.now()
      counts = inside_circle(n_samples)
      my_pi = 4.0 * counts / n_samples
      end_time = datetime.datetime.now()
      elapsed_time = (end_time - start_time).total_seconds()
      size_of_float = np.dtype(np.float64).itemsize
      memory_required = 3 * n_samples * size_of_float / (1024**3)
      abs_err = abs(my_pi - math.pi)
      rel_err = abs(abs_err / math.pi) / abs(math.pi)
      grid = np.vstack([grid, [n_samples, rel_err, memory_required, elapsed_time]])
    print("# of Samples: {}, Pi: {}, memory: {} GiB, time: {} s".format(
          n_samples, my_pi, memory_required, elapsed_time))
    np.savetxt('matplotlibProject.csv', grid, delimiter=',')

# of Samples: 14999, Pi: 3.1335422361490766, memory: 0.0003352537751197815 GiB, time: 0.000437 s


In [39]:
data = np.loadtxt(fname='matplotlibProject.csv',
delimiter=',')
n_val=data[ : , 0 ]
err_val=data[ : , 1]
mem_val=data[ : , 2]
time_val=data[ : , 3]